# Ajustarea fină a modelelor Open AI

Acest caiet este bazat pe ghidul actual furnizat în documentația [Fine Tuning](https://platform.openai.com/docs/guides/fine-tuning?WT.mc_id=academic-105485-koreyst) de la Open AI.

Ajustarea fină îmbunătățește performanța modelelor fundamentale pentru aplicația ta prin reantrenarea acestora cu date suplimentare și context relevant pentru acel caz de utilizare sau scenariu specific. Reține că tehnicile de inginerie a promptului precum _few shot learning_ și _retrieval augmented generation_ îți permit să îmbunătățești promptul implicit cu date relevante pentru a crește calitatea. Totuși, aceste abordări sunt limitate de dimensiunea maximă a ferestrei de tokeni a modelului fundamental vizat.

Prin ajustarea fină, practic reantrenăm modelul în sine cu datele necesare (permițându-ne să folosim mult mai multe exemple decât pot încăpea în fereastra maximă de tokeni) - și implementăm o versiune _personalizată_ a modelului care nu mai are nevoie să primească exemple la momentul inferenței. Acest lucru nu doar că îmbunătățește eficacitatea designului promptului nostru (avem mai multă flexibilitate în utilizarea ferestrei de tokeni pentru alte lucruri), dar potențial îmbunătățește și costurile noastre (prin reducerea numărului de tokeni pe care trebuie să îi trimitem modelului la inferență).

Ajustarea fină are 4 pași:
1. Pregătește datele de antrenament și încarcă-le.
1. Rulează jobul de antrenament pentru a obține un model ajustat fin.
1. Evaluează modelul ajustat fin și iterează pentru calitate.
1. Implementează modelul ajustat fin pentru inferență când ești mulțumit.

Reține că nu toate modelele fundamentale suportă ajustarea fină - [verifică documentația OpenAI](https://platform.openai.com/docs/guides/fine-tuning/what-models-can-be-fine-tuned?WT.mc_id=academic-105485-koreyst) pentru cele mai recente informații. De asemenea, poți ajusta fin un model care a fost deja ajustat fin anterior. În acest tutorial, vom folosi `gpt-35-turbo` ca model fundamental țintă pentru ajustarea fină.

---


### Pasul 1.1: Pregătiți-vă setul de date

Să construim un chatbot care să vă ajute să înțelegeți tabelul periodic al elementelor răspunzând la întrebări despre un element printr-un limerick. În _acest_ tutorial simplu, vom crea doar un set de date pentru a antrena modelul cu câteva exemple de răspunsuri care arată formatul așteptat al datelor. Într-un caz real, ar trebui să creați un set de date cu mult mai multe exemple. De asemenea, este posibil să puteți folosi un set de date deschis (pentru domeniul aplicației dvs.) dacă există unul, și să îl reformatați pentru utilizarea în fine-tuning.

Deoarece ne concentrăm pe `gpt-35-turbo` și căutăm un răspuns într-un singur tur (completare chat), putem crea exemple folosind [acest format sugerat](https://platform.openai.com/docs/guides/fine-tuning/preparing-your-dataset?WT.mc_id=academic-105485-koreyst) care reflectă cerințele completării chat OpenAI. Dacă vă așteptați la conținut conversațional multi-tur, ați folosi [formatul de exemplu multi-tur](https://platform.openai.com/docs/guides/fine-tuning/multi-turn-chat-examples?WT.mc_id=academic-105485-koreyst) care include un parametru `weight` pentru a semnala care mesaje ar trebui folosite (sau nu) în procesul de fine-tuning.

Vom folosi formatul mai simplu, cu un singur tur, pentru tutorialul nostru aici. Datele sunt în [format jsonl](https://jsonlines.org/?WT.mc_id=academic-105485-koreyst) cu 1 înregistrare pe linie, fiecare reprezentată ca un obiect JSON formatat. Fragmentul de mai jos arată 2 înregistrări ca exemplu - vedeți [training-data.jsonl](../../../../../18-fine-tuning/python/openai/training-data.jsonl) pentru setul complet de exemple (10 exemple) pe care îl vom folosi pentru tutorialul de fine-tuning. **Notă:** Fiecare înregistrare _trebuie_ definită într-o singură linie (nu împărțită pe mai multe linii, așa cum este tipic într-un fișier JSON formatat)

```json
{ "messages": [{"role": "system", "content": "Elle is a factual chatbot that answers questions about elements in the periodic table with a limerick"}, {"role": "user", "content": "Tell me about Gallium"}, {"role": "assistant", "content": "Gallium, oh gallium, so light - Melts in your hand, oh what a sight - At 86 degrees - Its liquid with ease - And in semiconductors, it's out of sight"}]}
{ "messages": [{"role": "system", "content": "Elle is a factual chatbot that answers questions about elements in the periodic table with a limerick"}, {"role": "user", "content": "Tell me about Hydrogen"}, {"role": "assistant", "content": "Hydrogen, the first in the line - The lightest of all, so divine - It's in water, you see - And in stars, it's the key - The universe's most common sign"}]}
```

Într-un caz real, veți avea nevoie de un set mult mai mare de exemple pentru rezultate bune - compromisul va fi între calitatea răspunsurilor și timpul/costurile pentru fine-tuning. Folosim un set mic pentru a putea finaliza rapid fine-tuning-ul și a ilustra procesul. Consultați [acest exemplu din OpenAI Cookbook](https://github.com/openai/openai-cookbook/blob/main/examples/How_to_finetune_chat_models.ipynb?WT.mc_id=academic-105485-koreyst) pentru un tutorial de fine-tuning mai complex.


---

### Pasul 1.2 Încarcă Setul de Date

Încarcă datele folosind API-ul Files [așa cum este descris aici](https://platform.openai.com/docs/guides/fine-tuning/upload-a-training-file). Reține că pentru a rula acest cod, trebuie să fi făcut următorii pași mai întâi:
 - Ai instalat pachetul Python `openai` (asigură-te că folosești o versiune >=0.28.0 pentru cele mai noi funcționalități)
 - Ai setat variabila de mediu `OPENAI_API_KEY` cu cheia ta API OpenAI
Pentru a afla mai multe, vezi [Ghidul de configurare](./../../../00-course-setup/02-setup-local.md?WT.mc_id=academic-105485-koreyst) oferit pentru curs.

Acum, rulează codul pentru a crea un fișier pentru încărcare din fișierul tău local JSONL.


In [24]:
from openai import OpenAI
client = OpenAI()

ft_file = client.files.create(
  file=open("./training-data.jsonl", "rb"),
  purpose="fine-tune"
)

print(ft_file)
print("Training File ID: " + ft_file.id)

FileObject(id='file-JdAJcagdOTG6ACNlFWzuzmyV', bytes=4021, created_at=1715566183, filename='training-data.jsonl', object='file', purpose='fine-tune', status='processed', status_details=None)
Training File ID: file-JdAJcagdOTG6ACNlFWzuzmyV


---

### Pasul 2.1: Creează jobul de ajustare fină cu SDK-ul


In [25]:
from openai import OpenAI
client = OpenAI()

ft_filejob = client.fine_tuning.jobs.create(
  training_file=ft_file.id, 
  model="gpt-3.5-turbo"
)

print(ft_filejob)
print("Fine-tuning Job ID: " + ft_filejob.id)

FineTuningJob(id='ftjob-Usfb9RjasncaZ5Cjbuh1XSCh', created_at=1715566184, error=Error(code=None, message=None, param=None), fine_tuned_model=None, finished_at=None, hyperparameters=Hyperparameters(n_epochs='auto', batch_size='auto', learning_rate_multiplier='auto'), model='gpt-3.5-turbo-0125', object='fine_tuning.job', organization_id='org-EZ6ag0n0S6Zm8eV9BSWKmE6l', result_files=[], seed=830529052, status='validating_files', trained_tokens=None, training_file='file-JdAJcagdOTG6ACNlFWzuzmyV', validation_file=None, estimated_finish=None, integrations=[], user_provided_suffix=None)
Fine-tuning Job ID: ftjob-Usfb9RjasncaZ5Cjbuh1XSCh


---

### Pasul 2.2: Verifică starea jobului

Iată câteva lucruri pe care le poți face cu API-ul `client.fine_tuning.jobs`:
- `client.fine_tuning.jobs.list(limit=<n>)` - Listează ultimele n joburi de fine-tuning
- `client.fine_tuning.jobs.retrieve(<job_id>)` - Obține detalii despre un job specific de fine-tuning
- `client.fine_tuning.jobs.cancel(<job_id>)` - Anulează un job de fine-tuning
- `client.fine_tuning.jobs.list_events(fine_tuning_job_id=<job_id>, limit=<b>)` - Listează până la n evenimente din job
- `client.fine_tuning.jobs.create(model="gpt-35-turbo", training_file="your-training-file.jsonl", ...)`

Primul pas al procesului este _validarea fișierului de antrenament_ pentru a te asigura că datele sunt în formatul corect.


In [26]:
from openai import OpenAI
client = OpenAI()

# List 10 fine-tuning jobs
client.fine_tuning.jobs.list(limit=10)

# Retrieve the state of a fine-tune
client.fine_tuning.jobs.retrieve(ft_filejob.id)

# List up to 10 events from a fine-tuning job
client.fine_tuning.jobs.list_events(fine_tuning_job_id=ft_filejob.id, limit=10)

SyncCursorPage[FineTuningJobEvent](data=[FineTuningJobEvent(id='ftevent-GkWiDgZmOsuv4q5cSTEGscY6', created_at=1715566184, level='info', message='Validating training file: file-JdAJcagdOTG6ACNlFWzuzmyV', object='fine_tuning.job.event', data={}, type='message'), FineTuningJobEvent(id='ftevent-3899xdVTO3LN7Q7LkKLMJUnb', created_at=1715566184, level='info', message='Created fine-tuning job: ftjob-Usfb9RjasncaZ5Cjbuh1XSCh', object='fine_tuning.job.event', data={}, type='message')], object='list', has_more=False)

In [30]:
# Once the training data is validated
# Track the job status to see if it is running and when it is complete
from openai import OpenAI
client = OpenAI()

response = client.fine_tuning.jobs.retrieve(ft_filejob.id)

print("Job ID:", response.id)
print("Status:", response.status)
print("Trained Tokens:", response.trained_tokens)

Job ID: ftjob-Usfb9RjasncaZ5Cjbuh1XSCh
Status: running
Trained Tokens: None


---

### Pasul 2.3: Urmărirea evenimentelor pentru monitorizarea progresului


In [44]:
# You can also track progress in a more granular way by checking for events
# Refresh this code till you get the `The job has successfully completed` message
response = client.fine_tuning.jobs.list_events(ft_filejob.id)

events = response.data
events.reverse()

for event in events:
    print(event.message)

Step 85/100: training loss=0.14
Step 86/100: training loss=0.00
Step 87/100: training loss=0.00
Step 88/100: training loss=0.07
Step 89/100: training loss=0.00
Step 90/100: training loss=0.00
Step 91/100: training loss=0.00
Step 92/100: training loss=0.00
Step 93/100: training loss=0.00
Step 94/100: training loss=0.00
Step 95/100: training loss=0.08
Step 96/100: training loss=0.05
Step 97/100: training loss=0.00
Step 98/100: training loss=0.00
Step 99/100: training loss=0.00
Step 100/100: training loss=0.00
Checkpoint created at step 80 with Snapshot ID: ft:gpt-3.5-turbo-0125:bitnbot::9OFWyyF2:ckpt-step-80
Checkpoint created at step 90 with Snapshot ID: ft:gpt-3.5-turbo-0125:bitnbot::9OFWyzhK:ckpt-step-90
New fine-tuned model created: ft:gpt-3.5-turbo-0125:bitnbot::9OFWzNjz
The job has successfully completed


### Pasul 2.4: Vizualizează starea în Panoul de control OpenAI


De asemenea, puteți vizualiza starea accesând site-ul OpenAI și explorând secțiunea _Fine-tuning_ a platformei. Aceasta vă va arăta starea jobului curent și vă va permite să urmăriți istoricul execuțiilor anterioare ale joburilor. În această captură de ecran, puteți vedea că execuția anterioară a eșuat, iar a doua rulare a reușit. Pentru context, acest lucru s-a întâmplat când prima rulare a folosit un fișier JSON cu înregistrări formatate incorect - odată corectat, a doua rulare s-a finalizat cu succes și a făcut modelul disponibil pentru utilizare.

![Fine-tuning job status](../../../../../translated_images/ro/fine-tuned-model-status.563271727bf7bfba.webp)


De asemenea, puteți vizualiza mesajele de stare și metricile derulând mai jos în tabloul de bord vizual, așa cum este prezentat:

| Mesaje | Metrici |
|:---|:---|
| ![Mesaje](../../../../../translated_images/ro/fine-tuned-messages-panel.4ed0c2da5ea1313b.webp) |  ![Metrici](../../../../../translated_images/ro/fine-tuned-metrics-panel.700d7e4995a65229.webp)|


---

### Pasul 3.1: Obțineți ID-ul și testați modelul ajustat în cod


In [46]:
# Retrieve the identity of the fine-tuned model once ready
response = client.fine_tuning.jobs.retrieve(ft_filejob.id)
fine_tuned_model_id = response.fine_tuned_model
print("Fine-tuned Model ID:", fine_tuned_model_id)

Fine-tuned Model ID: ft:gpt-3.5-turbo-0125:bitnbot::9OFWzNjz


In [47]:
# You can then use that model to generate completions from the SDK as shown
# Or you can load that model into the OpenAI Playground (in the UI) to validate it from there.
from openai import OpenAI
client = OpenAI()

completion = client.chat.completions.create(
  model=fine_tuned_model_id,
  messages=[
    {"role": "system", "content": "You are Elle, a factual chatbot that answers questions about elements in the periodic table with a limerick"},
    {"role": "user", "content": "Tell me about Strontium"},
  ]
)
print(completion.choices[0].message)

ChatCompletionMessage(content="Strontium, a metal so bright - It's in fireworks, a dazzling sight - It's in bones, you see - And in tea, it's the key - It's the fortieth, so pure, that's the right", role='assistant', function_call=None, tool_calls=None)


---

### Pasul 3.2: Încarcă și testează modelul ajustat în Playground

Acum poți testa modelul ajustat în două moduri. Mai întâi, poți vizita Playground și folosi meniul derulant Models pentru a selecta modelul tău nou ajustat din opțiunile listate. Cealaltă opțiune este să folosești opțiunea "Playground" afișată în panoul Fine-tuning (vezi captura de ecran de mai sus) care lansează următoarea vizualizare _comparativă_ ce arată versiunile modelului de bază și a celui ajustat unul lângă altul pentru o evaluare rapidă.

![Fine-tuning job status](../../../../../translated_images/ro/fine-tuned-playground-compare.56e06f0ad8922016.webp)

Completează pur și simplu contextul sistemului folosit în datele tale de antrenament și oferă întrebarea ta de test. Vei observa că ambele părți sunt actualizate cu același context și întrebare. Rulează comparația și vei vedea diferența în răspunsurile dintre ele. _Observă cum modelul ajustat redă răspunsul în formatul pe care l-ai furnizat în exemplele tale, în timp ce modelul de bază urmează pur și simplu promptul sistemului_.

![Fine-tuning job status](../../../../../translated_images/ro/fine-tuned-playground-launch.5a26495c983c6350.webp)

Vei observa că comparația oferă și numărul de tokeni pentru fiecare model, precum și timpul necesar pentru inferență. **Acest exemplu specific este unul simplist menit să arate procesul, dar nu reflectă un set de date sau un scenariu real**. Poți observa că ambele mostre arată același număr de tokeni (contextul sistemului și promptul utilizatorului sunt identice), iar modelul ajustat necesită mai mult timp pentru inferență (model personalizat).

În scenarii reale, nu vei folosi un exemplu simplu ca acesta, ci vei face ajustări pe date reale (de exemplu, catalog de produse pentru serviciul clienți) unde calitatea răspunsului va fi mult mai evidentă. În _acel_ context, obținerea unei calități echivalente a răspunsului cu modelul de bază va necesita o inginerie de prompt personalizată mai complexă, ceea ce va crește utilizarea tokenilor și, potențial, timpul de procesare pentru inferență. _Pentru a încerca acest lucru, consultă exemplele de fine-tuning din OpenAI Cookbook pentru a începe._

---


---

<!-- CO-OP TRANSLATOR DISCLAIMER START -->
**Declinare de responsabilitate**:  
Acest document a fost tradus folosind serviciul de traducere AI [Co-op Translator](https://github.com/Azure/co-op-translator). Deși ne străduim pentru acuratețe, vă rugăm să rețineți că traducerile automate pot conține erori sau inexactități. Documentul original în limba sa nativă trebuie considerat sursa autorizată. Pentru informații critice, se recomandă traducerea profesională realizată de un specialist uman. Nu ne asumăm responsabilitatea pentru eventualele neînțelegeri sau interpretări greșite rezultate din utilizarea acestei traduceri.
<!-- CO-OP TRANSLATOR DISCLAIMER END -->
